In [1]:
from __future__ import print_function

import torch
import torch.nn as nn

## What happens in your CNN: conv2d
### (discrete 2-D convolutions)
This short gist assumes some basic knowledge about fully-connected neural networks - the kind everyone is first introduced to. Here we try to understand in detail what happens when using a 2-D convolutional layer in our neural networks. Convolutional layers and CNNs in general are very popular in domains like Computer Vision, but can also be applied to many other types of data with an intrinsic structure (e.g. sound clips or text).  

### Why?
One might ask why should we even bother with convolutions in the first place?  
In short: With discrete convolutions we can exploit the *intrinsic structure*  of the given data (if present). We'll use image inputs as an example. Their intrinsic structure is obviously in two dimensions, therefore 2-D convolutions are useful in this case. Bonus: They are also the easiest to visualize.  
Another reason to use convolutions is due to the fact that images possess *localized concepts*. This means pixels close to each other share some correlations. They are likely to be similar, unless there's an edge at that location - which also is an important feature we don't want to miss. The same goes e.g. for text where words next to each other probably share the same context. 

### How do discrete convolutions work?
Discrete convolutions can basically be described as "a linear transformations which preserve ordering" ([from this nice paper on which a lot of this notebook is based](https://arxiv.org/pdf/1603.07285.pdf)).  
This is achieved by a **kernel** of a given size (here 2-dimensional *height x width*) sliding across the **input feature map** (here one color channel of the image). At each **stride** the product between each element of the kernel and the input element it overlaps is computed and all elements are summed up to obtain the output of the current stride. This process is repeated stride by stride to produce **output feature maps**.  
Note that for a 3-D convolution the kernel would simply be a cuboid sliding across height, width and depth. The 1-D case is now rather obvious. But the  effect for all cases is the same: The convolution preserves the intrinsic structure of the input.
<img src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/no_padding_no_strides.gif">
Here the blue map represents the input and the cyan map the output feature map.  
Any kernel size other than `(1, 1)` will results in smaller output feature maps compared to the input. As can be seen above the kernel size of `(3, 3)` turns the `(4, 4)` input feature map into a `(2, 2)` output. If we want the input and output feature maps to be of the same size we can simply *pad* the input with zeros.  
For example a `(5, 5)` input feature map with a **zero-padding** of 1 pixel for each dimension preserves the spatial size of the input even with the kernel size of `(3, 3)`.
<img src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/same_padding_no_strides.gif">
The reasons why we may want to preserve the spatial size of the input are manifold:
* designing networks is easier since the tensor dimensions will simply fit
* allows for deeper networks (without padding the size will be reduced too quickly)
* can improve performance by keeping information at the borders
* some newer architectures need to concatenate convolutional layers with  
`(1, 1)`, `(3, 3)` and `(5, 5)` kernels, which wouldn't be possible without padding since the dimensions wouldn't match (see [inception module](https://i.stack.imgur.com/ldTdM.png) for example)  

We can also choose to have the kernel move across the input with larger steps if we use a larger **stride**. As can be seen in the example below with a kernel size of `(3, 3)` and a stride of `(2, 2)`, larger strides quickly reduce the spatial size of the input and result in less computations per convolution compared to smaller strides, but also leads to information loss. So there's a trade-off.
<img src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/no_padding_strides.gif">

### Where are the trainable weights in convolutional layers?
Comparing convolutional layers to their 'classical' counterpart of fully-connected layers instead of having trainable weight matrices we have kernels sliding across the input. The question may arise: Where are the trainable weights in this case?  
For convolutional layers the weights are contained in the collection of kernels we use. Biases aren't treated any different from the way they are used for fully-connected layers.  
So a `(5, 5)` kernel contains 25 weights. That aren't too many! After all, a big advantage of convolutional layers is that weights are applied to multiple locations in the input and therefore they get by with a much smaller number of weights.  
But usually we do not noly have one kernel, but rather many more depending on how many output feature maps we wish to create and how many input feature maps we have at hand. The choice of the number of output feature maps of a convolutional layer can be compared to the choice of the number of hidden nodes in a fully-connected layer.  
To see how this all comes together let's take a look at the formula given in the [PyTorch docs](http://pytorch.org/docs/master/nn.html#conv2d) on `torch.nn.Conv2d`:
$$ \text{out}(N_i, C_{\text{out}, j}) = \text{bias}(C_{\text{out}, j}) 
+ \sum_{k=0}^{C_{\text{in}-1}} \text{weight}(C_{\text{out}, j}, k) \star \text{input}(N_i, k)$$
This formula maps our input tensor [$N, C_\text{in}, H, W$] to our output tensor [$N, C_\text{out}, H_\text{out}, W_\text{out}$]. The $\star$ operator can be seen as the operation of sliding the kernel across the input feature map (recall there's another summation happening in this operation as described above). $N$ simply describes the number of inputs (also known as batch size) to take into account for the optimization step. Since this is not central to understanding convolutional layers N can be put aside for this consideration. $C$ denotes the respective feature map, $\text{weight}$ is our kernel and $\text{bias}$ should be self-explanatory.  
From this we can quickly infer that we have $C_\text{out} \cdot C_\text{in}$ kernels.

TODO:
* (maybe describe dilation - but seems rather advanced compared to the 'core' features of convolutional layers)
* how many kernels $\checkmark$
* how many trainable weights?
* how is this different from a fully-connected layer?
    * how can it be compared?
    
* compare performance for 1 hidden layer networks on MNIST
    * conv2d with (5, 5) kernel and 1 stride, no padding - 25 weights
    * fully-connected from flattened (28x28=784) to flattened(24x24=576) (451584 weights)
    * both with a fc output layer (10)